<a href="https://colab.research.google.com/github/HYLee810/projectmanager/blob/main/2_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_04_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91%ED%95%98%EA%B8%B0_%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%BD%94%EB%9E%A9%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 분석

## 4. 웹 크롤링으로 데이터 수집

---------------------

### 01.웹 크롤링의 이해
- **웹 크롤링(Web Crawling)**: 웹 페이지로부터 원하는 정보를 추출하는 기법, 보통 여러 페이지 탐색하는 방법,
- **웹 스크래핑(Web Scraping)**: 특정한 하나의 웹 페이지를 탐색하는 방법
- **웹 크롤링과 웹 스크래핑의  방법이 동일하여 보통 웹 크롤링이라고 통칭해서 부른다.**

### 라이브러리 설치하기

In [1]:
# 정적 크롤링을 위한 requests 설치
!pip install requests

In [2]:
# HTML과 XML 문서를 파싱하기 위한 파이썬 패키지
!pip install beautifulsoup4

In [3]:
# 동적 크롤링을 위한 셀레니움 설치
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


### 라이브러리 확인하기

In [4]:
import requests

requests.get("https://www.naver.com") #IDC(internet data center)(네이버와 같은)

<Response [200]>

In [9]:
import requests

response = requests.get("https://google.com") #response는 변수

# 응답 상태
print('#응답 상태: ', response.status_code) # 200은 국제코드로 동일

# 응답 바이너리 원문
print('#응답 바이너리 원문: ', response.content)

# 응답 UTF-8로 인코딩된 문자열
# print('#응답 UTF-8로 인코딩된 문자열: ', response.text) # 웹페이지 코드 구글에서 F12를 누르면 웹코드 확인과 동일

# 응답 헤더
print('#응답 헤더: ', response.headers)

# # 응답 헤더: 콘텐트 유형
print('#응답 헤더유형: ', response.headers['Content-Type'])

#응답 상태:  200
#응답 바이너리 원문:  b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content="Search the world\'s information, including webpages, images, videos and more. Google has many special features to help you find exactly what you\'re looking for." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="jTbgXwJJsE87ZnAJJXbvxA">(function(){var _g={kEI:\'_bDuZPmCHrTf0PEPzbqc8As\',kEXPI:\'0,1359408,6060,206,4804,2316,383,246,5,1129120,1197704,187,510,380090,16114,28684,22431,1361,12313,4751,12835,4998,17075,38444,2872,2891,3926,4423,3405,606,30668,30022,2614,12710,2025,1,16916,2652,4,3832,29062,13065,11443,2215,2980,24070,6624,7596,1,42154,2,16395,342,3534,19490,5679,1021,31121,4569,6258,23418,1252,25135,5017,2912,2,2,1,26632,8155,7381,2,3,15965,872,78

-------------------------------

## 02.정적 크롤링(스크래핑)

### [실습] 네이버 뉴스 검색 데이터 수집하기
단, 네이버에서는 한번에 90개까지만 수집 가능한 상태임 <br>
- **1.데이터 가져올 웹페이지**: 네이버 뉴스검색(검색어:우영우) https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%9A%B0%EC%98%81%EC%9A%B0
- **2.가져올 정보**: 기사제목, URL
- **3.DataFrame으로 만들기**
- **4.DataFrame --> csv파일로 만들기**

In [11]:
#VSC에서 실행
import requests
from bs4 import BeautifulSoup
import pandas as pd

print('-'*50)
print("#네이버 뉴스 데이터 크롤링하기...")
print('-'*50)

# 검색 URL 지정하기
# query=검색어, start=페이지 (10건씩 추출)
news_url = 'https://search.naver.com/search.naver?where=news&query={}&start=' # {}안에 키워드

# 검색어 직접 입력 받기
keyword = input('검색 키워드를 입력하세요 : ')
query = keyword.replace(' ', '+') # 검색에서 공백문자를 +로 대체해 사용하도록 적용
print(f'query: {query}')

def get_naverNewsTitle(total=100, start_cnt=1):

    # 1.웹 데이터 수집하기
    titles, urls= [],[]     # 제목=[], url=[] 데이터담기
    while True:

        # 웹 페이지 요청하기
        print(f'요청 query:{news_url.format(query) + str(start_cnt)} ')
        rep = requests.get(news_url.format(query) + str(start_cnt)) # 결과값


        # 2.응답 결과 파싱하기(제목, url 추출하기)
        soup = BeautifulSoup(rep.text, 'html.parser')
#         page_tags   = soup.select('ul.list_news > li > div > div > a')
        page_tags   = soup.select('div.news_area > a') # 여기에서 정보 파싱
        page_titles = list(map(lambda tag: tag.get('title'), page_tags))
        page_urls   = list(map(lambda tag: tag.get('href'), page_tags))

        # 목적 개수만큼 추출하기
        if len(page_titles) <= 0:
            break
        elif start_cnt >= total:
            break
        else:
            titles += page_titles
            urls += page_urls
            start_cnt += len(page_titles)

        # 데이터 수집 진행률 표시
        print(f"{int( start_cnt-1/total*100 ) }% ", end="")

    print()
    print(f"검색어[{keyword}] 로 검색된 뉴스 [{len(titles)}]건 가져오기 완료!")
    print('-'*50)

    # 3.titles, urls 2차원 리스트로 만들어 DataFrame으로 만든다.
    datas = [[t, u] for t, u in zip(titles, urls)]
    df = pd.DataFrame(datas, columns=['title','url'])
    return df

df = get_naverNewsTitle(10, 1)  # df=dataframe
df

# 4.파일로 저장하기
file = f"./sample_data/naver_news_{keyword}_title.csv"
news_df = df['title'] #제목만
news_df.to_csv(file, index=False, encoding="utf-8-sig")
df

--------------------------------------------------
#네이버 뉴스 데이터 크롤링하기...
--------------------------------------------------
검색 키워드를 입력하세요 : 손흥민
query: 손흥민
요청 query:https://search.naver.com/search.naver?where=news&query=손흥민&start=1 

검색어[손흥민] 로 검색된 뉴스 [0]건 가져오기 완료!
--------------------------------------------------


,title,url


### [실습] (수집된 데이터)기사 제목의 단어(토큰) 빈도수 분석하기
- ** CountVectorizer 클래스 사용 : 단어 빈도수 추출**
- **단, 한글에서 불용어 처리 및 가중치 처리 등 자연어 텍스트 전처리에 필요한 여러 가지 방법은 여기서 다루지 않는다.**

In [13]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 코랩에서 위 코드를 실행시킨 후  반드시 코랩(Colab)의 런타임을 재시작 합니다.

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 16 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (9,494 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120831 files and dire

In [1]:
#[코랩용 한글]
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')        # 방법1
plt.rcParams['font.family'] = 'NanumBarunGothic' # 방법2

In [2]:
!pip install scikit-learn

In [3]:
import sklearn
sklearn.__version__

'1.2.2'

In [4]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumBarunGothic'


def get_wordTokenCount(corpus):
    from sklearn.feature_extraction.text import CountVectorizer

    # 기사제목을 토큰화
    vect = CountVectorizer().fit(corpus)
    count = vect.transform(corpus).toarray().sum(axis=0)

    # 토큰 빈도수로 정렬하고 토큰명 추출
    idx = np.argsort(-count)  # 내림 정렬하여 인덱스 반환: 토큰의 인덱스
    count = count[idx]        # 토큰의 빈도수
    feature_name = np.array(vect.get_feature_names_out())[idx]  # 토큰값

    # 빈도수 많은 순서대로 토큰명 10개만 출력
    print(list(zip(feature_name, count))[:10])

    return feature_name, count


def draw_wordTokenCountGraph(data, freq):
    plt.bar(data, freq)
    plt.grid()
    plt.show()

    # 그래프 그림 저장히기
    plt.savefig(f'image/{keyword}_bar_graph.png')


# 기사제목을 <<말뭉치>>로 사용
corpus = df['title'].to_list()
# print(corpus)

# 기사제목을 토큰화하여 빈도수 가져오기
feature_name, count = get_wordTokenCount(corpus)

# 단어(토큰) 빈도 수_Bar그래프 그리기 : 상위 10개
draw_wordTokenCountGraph(feature_name[:10], count[:10])


NameError: ignored

### [실습] (수집된 데이터)기사 제목의 토큰 빈도수 워드클라우드로 시각화
- 워드클라우드 입력데이터 : 딕셔너리 타입

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,6)

# 자신의 컴퓨터 환경에 맞는 한글 폰트 경로를 설정(window에서)
# font_path = 'malgun'  # C:/Windows/Fonts/
font_path = 'HMKMMAG' # C:/Windows/Fonts/HMKMMAG.TTF

# (토큰명, 빈도수) 딕셔너리 타입으로 변환
data = dict(zip(feature_name, count))

# 워드클라우드로 그래프로 시각화
wc = WordCloud(width = 1000, height = 600, background_color="white", font_path=font_path)
plt.imshow(wc.generate_from_frequencies(data)) #딕셔너리
plt.axis("off")
plt.show()

# 파일로 저장하기
wc.to_file(f'image/{keyword}_워드클라우드.png')

### [실습] (수집된 데이터)기사 제목의 토큰 빈도수 출력하기

In [ ]:
data = dict(zip(feature_name, count))
token_df = pd.DataFrame(data, index=[1])
token_df

# token_df.to_csv('data/token.csv', index=False, encoding="utf-8-sig")

-----

## 03.동적 크롤링(PC에서 실행)

- https://chromedriver.chromium.org/
- 최신 chrome webdriver 다운로드
- 해당 위치에 WebDriver폴더 만들고 exe파일 옮겨놓는다.
-  (C:/python/projectmanager/WebDriver/chromedriver.exe)
- 자신의 크롬 웹 브라우저의 버전을 확인하고 버전에 맞는 것을 다운로드해야한다.

#### # webdriver 동작 테스트하기
- 자신의 크롬 웹 브라우저의 버전을 확인하고 버전에 맞는 것을 다운로드해야한다. 그렇지 않으면 오류가 발생한다.
- 아래 코드를 실행시키면 크롬 브라우져가 잠깐 실행되었다 닫힌다.

In [ ]:
from selenium import webdriver

# chromedriver.exe 파일이 있는 경로
driver = './WebDriver/chromedriver.exe'
wd = webdriver.Chrome(driver)

wd.get('https://www.naver.com/')

wd.close()  # 브라우저가 실행되었다가 자동으로 닫힌다.

### [실습]  커피빈매장 정보 크롤링하여 파일로 저장하기

- 아래 사이트를 이용해 호출해야할 자바스크립트 함수를 확인하다.
- https://www.coffeebeankorea.com
- https://www.coffeebeankorea.com/store/store.asp
- (매장 번호로) 자세히보기: javascript:storePop2('374');

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime

from selenium import webdriver
import time

MAX = 10     # 추출 데이터 건수

#[매장 추출 함수]
def getStoreInfo():
    CoffeeBean_URL = "https://www.coffeebeankorea.com/store/store.asp"
    wd = webdriver.Chrome('./WebDriver/chromedriver.exe')

    result = []  # 데이터 저장 변수
    total, cnt = 370, 0
    for i in range(1, total+1):  #매장 수 만큼(370) 반복
        wd.get(CoffeeBean_URL)
        time.sleep(1)  #웹페이지 연결할 동안 1초 대기
        try:
            wd.execute_script("storePop2(%d)" %i)
            time.sleep(1) #스크립트 실행 할 동안 1초 대기

            html = wd.page_source
            soupCB = BeautifulSoup(html, 'html.parser')
            store_name_h2 = soupCB.select("div.store_txt > h2")
            store_name = store_name_h2[0].string  #매장 이름

            store_info = soupCB.select("div.store_txt > table.store_table > tbody > tr > td")
            store_address_list = list(store_info[2])
            store_address = store_address_list[0]  #매장 주소

            store_phone = store_info[3].string     #매장 전화번호
            result.append([store_name]+[store_address]+[store_phone])
            cnt += 1
            # 매장정보 가져온 데이터 출력하기
            print("[%3d] %3d - %s" % (cnt, i, store_name))


             # MAX값에 해당하는 건수 만큼만 실행하기
            if cnt >= MAX:
                break

        except:
            continue

    return result

#---------------
# main
#---------------
print('CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
result = getStoreInfo()  #[매장 추출 함수]호출하기

df_cb = pd.DataFrame(result, columns=('store', 'address','phone'))
df_cb.head()

In [ ]:
# 파일로 저장하기
file = './data/CoffeeBean_매장정보.csv'
df_cb.to_csv(file, encoding='cp949', mode='w', index=False)

df = pd.read_csv(file, encoding='cp949')
df.head()

--------------

끝